## Conversion from convolution filter to binary filter

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers


2021-12-20 03:02:43.353852: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-20 03:02:43.353903: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
assert tf.__version__ >= "2.7.0"

In [3]:
tf.config.run_functions_eagerly(True)
tf.executing_eagerly()
tf.data.experimental.enable_debug_mode()


In [4]:
# Model / data parameters
num_classes = 20
input_shape = (32, 32, 3)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data(label_mode='coarse')

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to one-hot representations
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [5]:
@tf.custom_gradient
def generate_B(W,u): # generate B given weight W and a specified u
  def grad(upstream): # define gradient by ourselves (dc/dW, dc/du)
    dB_dW = 1
    return upstream * dB_dW, 1
  std_W = tf.math.reduce_std(W) # calculate std(W)
  mean_W =  tf.math.reduce_mean(W) # calculate mean(W)
  Bi = tf.math.sign(W - mean_W + u * mean_W) #calculate Bi
  return Bi, grad

In [6]:
from sklearn.linear_model import LinearRegression
def generate_alpha(W,list_of_B): # generate alpha given weight W and list of B containing all Bi
  W = tf.reshape(W, [-1]).numpy() #vectorize W
  list_of_reshape_B = []
  for i in list_of_B: # vectorize Bi
    list_of_reshape_B.append(tf.reshape(i, [-1]))
  B = tf.stack(list_of_reshape_B, axis=1).numpy() # put Bi together into a large matrix
  reg = LinearRegression(fit_intercept=False).fit(B, W) # linear regression
  alpha = reg.coef_ # coefficient of linear regression, which is alpha
  return alpha



In [7]:
@tf.custom_gradient
def H_v(R, v): # calculate H_v given input R and shift parameter v
  def grad(upstream):  # define gradient by ourselves (dc/dR, dc/dv)
    return upstream, tf.math.reduce_sum(upstream)
  return 2 * tf.cast(tf.math.greater(tf.math.add(R, v), tf.constant(0.5)), tf.float32) - 1, grad # calculate H_v

Initialization of ABCConv class with M=5,N=5

In [8]:
import tensorflow as tf
import tensorflow.keras as keras
import keras.layers as layers
import numpy as np

# number of binary activations
N = 5
class ABCConv(layers.Conv2D): # subclass of layers.Conv2D
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.betas = self.add_weight(
            shape=(N,), initializer="random_normal", trainable=True
        )
        self.vs = self.add_weight(shape=(N,), initializer="random_normal", trainable=True)

    def convolution_op(self, inputs, kernel): # override convolution_op
        ## Multiple binary activations and bitwise convolution
        
        # given beta's and v's, get all binary activations
        binary_activations = []
        list_of_A = []
        for i in range(0,N):
          list_of_A.append(H_v(inputs,self.vs[i]))
        

        # get all B
        M = 5
        list_of_u = []
        for i in range(1,M+1): # all u
          list_of_u.append(-1 + (i -1.0) * 2 / (M-1))
        list_of_B = []
        for i in range(1,M+1): # all B
          list_of_B.append(generate_B(kernel, list_of_u[i-1]))
       
        
        
        # get alphas 
        alphas = generate_alpha(kernel,list_of_B)
        # call individual convs
        convs = []
        for j in range(N):
          for i in range(M):
            convs.append(alphas[i]* self.betas[j] * tf.nn.conv2d(
                list_of_A[j],
                list_of_B[i],
                padding="VALID",
                strides=list(self.strides),
                name=self.__class__.__name__ + "conv" + str(i) + str(j),
              ))
        return tf.add_n(convs) # linear combination of all convolution results
        

In [9]:
model_LeNet_5_5 = keras.Sequential(
    [
        keras.Input(shape=input_shape),

        
        ABCConv(6, kernel_size=(5, 5), activation=None),

        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.BatchNormalization(),
        

        ABCConv(16, kernel_size=(5, 5), activation=None),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.BatchNormalization(),
        
        
        
        layers.Flatten(),
        layers.Dense(512, activation="relu"),
        layers.Dense(128, activation="relu"),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model_LeNet_5_5.summary()

2021-12-20 03:02:56.303955: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-12-20 03:02:56.303994: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-12-20 03:02:56.304016: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (cluster-99b4-m): /proc/driver/nvidia/version does not exist
2021-12-20 03:02:56.304289: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 abc_conv (ABCConv)          (None, 28, 28, 6)         466       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 6)        0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 14, 14, 6)        24        
 ormalization)                                                   
                                                                 
 abc_conv_1 (ABCConv)        (None, 10, 10, 16)        2426      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 16)         0         
 2D)                                                             
                                                        

In [ ]:
batch_size = 64
epochs = 5

model_LeNet_5_5.compile(loss="categorical_crossentropy", optimizer="adam", metrics=[tf.keras.metrics.CategoricalAccuracy(),tf.keras.metrics.TopKCategoricalAccuracy(k=3)],run_eagerly=True)

model_LeNet_5_5.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

2021-12-20 03:03:04.441987: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 552960000 exceeds 10% of free system memory.


Epoch 1/5
704/704 [==============================] - 456s 648ms/step - loss: 2.5415 - categorical_accuracy: 0.2224 - top_k_categorical_accuracy: 0.4452 - val_loss: 2.6852 - val_categorical_accuracy: 0.2082 - val_top_k_categorical_accuracy: 0.4180
Epoch 2/5
704/704 [==============================] - 458s 650ms/step - loss: 2.3200 - categorical_accuracy: 0.2871 - top_k_categorical_accuracy: 0.5308 - val_loss: 2.6635 - val_categorical_accuracy: 0.1866 - val_top_k_categorical_accuracy: 0.4038
Epoch 3/5
704/704 [==============================] - 458s 650ms/step - loss: 2.2272 - categorical_accuracy: 0.3165 - top_k_categorical_accuracy: 0.5634 - val_loss: 2.5359 - val_categorical_accuracy: 0.2678 - val_top_k_categorical_accuracy: 0.4982
Epoch 4/5
704/704 [==============================] - 456s 647ms/step - loss: 2.1439 - categorical_accuracy: 0.3383 - top_k_categorical_accuracy: 0.5917 - val_loss: 2.3349 - val_categorical_accuracy: 0.2888 - val_top_k_categorical_accuracy: 0.5296
Epoch 5/5
70

In [12]:

# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model_LeNet_5_5.evaluate(x_test, y_test, batch_size=128)
print("test loss, test acc:", results)

Evaluate on test data
79/79 [==============================] - 27s 339ms/step - loss: 2.5097 - categorical_accuracy: 0.2820 - top_k_categorical_accuracy: 0.5266
test loss, test acc: [2.509688377380371, 0.28200000524520874, 0.5266000032424927]
